In [1]:
# Fits a polynomial over the peak between start_range and end_range. Uses the local maxima of the second 
# derivative of that polynomial to determine the cutoff points for the peak itself. Fits a polynomial to 
# the data that excludes the peak, and subtracts that background polynomial from the spectrum data to 
# give final peak

# lorentzian fit to peak to give heights

import pylab as py
import os
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore', np.RankWarning)

def get_ratio(xdata, ydata, start_G_range, end_G_range, start_G_peak, start_2D_range, end_2D_range, start_2D_peak):
    
    x_np = np.array(xdata)
    y_np = np.array(ydata)

    # select G peak
    x_G = x_np[(np.argmax(x_np>start_G_range)):(np.argmax(x_np>end_G_range))]
    y_G = y_np[(np.argmax(x_np>start_G_range)):(np.argmax(x_np>end_G_range))]

    # select 2D peak
    x_2D = x_np[(np.argmax(x_np>start_2D_range)):(np.argmax(x_np>end_2D_range))]
    y_2D = y_np[(np.argmax(x_np>start_2D_range)):(np.argmax(x_np>end_2D_range))]



    # POLYNOMIAL FIT TO SPECTRUM

    # degree of polynomial fit
    degree_G = 10
    degree_2D = 10

    # calculate polynomial
    z_G = np.polyfit(x_G, y_G, degree_G)
    f_G = np.poly1d(z_G)
    z_2D = np.polyfit(x_2D, y_2D, degree_2D)
    f_2D = np.poly1d(z_2D)

    # calculate y values for polynomial
    y_G_poly = f_G(x_G)
    y_2D_poly = f_2D(x_2D)


    # derivatives
    import scipy.misc as sm
    derivs2_G = []
    for i in x_G:
        derivs2_G.append((sm.derivative(f_G, i, n=2)))
    derivs2_2D = [] 
    for i in x_2D:
        derivs2_2D.append((sm.derivative(f_2D, i, n=2)))



    # FIND LOCAL MAXIMA OF SECOND DERIVATIVE

    # fits a polynomial to 2nd derivative, finds maxima of that

    # degree of polynomial fit
    degree2_G = 10
    degree2_2D = 10

    # calculate polynomial
    z2_G = np.polyfit(x_G, derivs2_G, degree2_G)
    f2_G = np.poly1d(z2_G)
    z2_2D = np.polyfit(x_2D, derivs2_2D, degree_2D)
    f2_2D = np.poly1d(z2_2D)

    # calculate y values for polynomial
    y2_G = f2_G(x_G)
    y2_2D = f2_2D(x_2D)

    # find local maxima 
    from scipy.signal import argrelextrema

    # use start_peak to eliminate edge effects
    idx_G = np.argmax(x_G>start_G_peak)
    x_Gpeak = x_G[idx_G:]
    derivs2a_G = np.array(y2_G[idx_G:])

    idx_2D = np.argmax(x_2D>start_2D_peak)
    x_2Dpeak = x_2D[idx_2D:]
    derivs2a_2D = np.array(y2_2D[idx_2D:])

    # this is indexed from the start of x_G, x_2D
    maxs_G = argrelextrema(derivs2a_G, np.greater)
    start_idx_G = maxs_G[0][0] + idx_G
    end_idx_G = maxs_G[0][1] + idx_G
    start_G = x_G[start_idx_G]
    end_G = x_G[end_idx_G]

    maxs_2D = argrelextrema(derivs2a_2D, np.greater)
    start_idx_2D = maxs_2D[0][0] + idx_2D
    end_idx_2D = maxs_2D[0][1] + idx_2D
    start_2D = x_2D[start_idx_2D]
    end_2D = x_2D[end_idx_2D]




    # FIT CURVE TO BACKGROUND 

    # exclude peak
    x_G_no_peak = np.concatenate((x_G[:start_idx_G], x_G[end_idx_G:]))
    y_G_no_peak = np.concatenate((y_G[:start_idx_G], y_G[end_idx_G:]))

    x_2D_no_peak = np.concatenate((x_2D[:start_idx_2D], x_2D[end_idx_2D:]))
    y_2D_no_peak = np.concatenate((y_2D[:start_idx_2D], y_2D[end_idx_2D:]))

    # degree of polynomial fit to background
    degree3_G = 3
    degree3_2D = 3

    # calculate polynomial
    z3_G = np.polyfit(x_G_no_peak, y_G_no_peak, degree3_G)
    f3_G = np.poly1d(z3_G)

    z3_2D = np.polyfit(x_2D_no_peak, y_2D_no_peak, degree3_2D)
    f3_2D = np.poly1d(z3_2D)



    # SUBTRACT OUT BACKGROUND
    y_G_fixed = y_G - f3_G(x_G)
    y_2D_fixed = y_2D - f3_2D(x_2D)
    
    
    
    # CURVE FITTING
    from scipy.optimize import curve_fit
    from math import pi

    xdata_G = x_G
    ydata_G = y_G_fixed
    xdata_2D = x_2D
    ydata_2D = y_2D_fixed

    # initial parameters (guess - doesn't really matter)
    init_center_G = 1600
    init_width_G = 10
    init_amplitude_factor_G = 300

    init_center_2D = 2700
    init_width_2D = 10
    init_amplitude_factor_2D = 300

    init_params_G = [init_center_G, init_width_G, init_amplitude_factor_G]
    init_params_2D = [init_center_2D, init_width_2D, init_amplitude_factor_2D]

    # lorentzian peak function
    def lorentz(x, center, width, A):
        return ( A * (0.5 * width) / (pi * ( ((x - center) ** 2) + ((0.5 * width) ** 2) ) ))

    params_G, cov_G = curve_fit(lorentz, xdata_G, ydata_G, p0=init_params_G)
    height_G = params_G[2] * 2 / (pi * params_G[1])
    FWHM_G = params_G[1]

    params_2D, cov_2D = curve_fit(lorentz, xdata_2D, ydata_2D, p0=init_params_2D)
    height_2D = params_2D[2] * 2 / (pi * params_2D[1])
    FWHM_2D = params_2D[1]

    ratio = height_2D / height_G

    return([ratio, height_G, height_2D])

In [ ]:
def get_ratio_with_plots(xdata, ydata, start_G_range, end_G_range, start_G_peak, start_2D_range, end_2D_range, start_2D_peak):
    
    x_np = np.array(xdata)
    y_np = np.array(ydata)

    # select G peak
    x_G = x_np[(np.argmax(x_np>start_G_range)):(np.argmax(x_np>end_G_range))]
    y_G = y_np[(np.argmax(x_np>start_G_range)):(np.argmax(x_np>end_G_range))]

    # select 2D peak
    x_2D = x_np[(np.argmax(x_np>start_2D_range)):(np.argmax(x_np>end_2D_range))]
    y_2D = y_np[(np.argmax(x_np>start_2D_range)):(np.argmax(x_np>end_2D_range))]



    # POLYNOMIAL FIT TO SPECTRUM

    # degree of polynomial fit
    degree_G = 10
    degree_2D = 10

    # calculate polynomial
    z_G = np.polyfit(x_G, y_G, degree_G)
    f_G = np.poly1d(z_G)
    z_2D = np.polyfit(x_2D, y_2D, degree_2D)
    f_2D = np.poly1d(z_2D)

    # calculate y values for polynomial
    y_G_poly = f_G(x_G)
    y_2D_poly = f_2D(x_2D)


    # derivatives
    import scipy.misc as sm
    derivs2_G = []
    for i in x_G:
        derivs2_G.append((sm.derivative(f_G, i, n=2)))
    derivs2_2D = [] 
    for i in x_2D:
        derivs2_2D.append((sm.derivative(f_2D, i, n=2)))



    # FIND LOCAL MAXIMA OF SECOND DERIVATIVE

    # fits a polynomial to 2nd derivative, finds maxima of that

    # degree of polynomial fit
    degree2_G = 10
    degree2_2D = 10

    # calculate polynomial
    z2_G = np.polyfit(x_G, derivs2_G, degree2_G)
    f2_G = np.poly1d(z2_G)
    z2_2D = np.polyfit(x_2D, derivs2_2D, degree_2D)
    f2_2D = np.poly1d(z2_2D)

    # calculate y values for polynomial
    y2_G = f2_G(x_G)
    y2_2D = f2_2D(x_2D)

    # find local maxima 
    from scipy.signal import argrelextrema

    # use start_peak to eliminate edge effects
    idx_G = np.argmax(x_G>start_G_peak)
    x_Gpeak = x_G[idx_G:]
    derivs2a_G = np.array(y2_G[idx_G:])

    idx_2D = np.argmax(x_2D>start_2D_peak)
    x_2Dpeak = x_2D[idx_2D:]
    derivs2a_2D = np.array(y2_2D[idx_2D:])

    # this is indexed from the start of x_G, x_2D
    maxs_G = argrelextrema(derivs2a_G, np.greater)
    start_idx_G = maxs_G[0][0] + idx_G
    end_idx_G = maxs_G[0][1] + idx_G
    start_G = x_G[start_idx_G]
    end_G = x_G[end_idx_G]

    maxs_2D = argrelextrema(derivs2a_2D, np.greater)
    start_idx_2D = maxs_2D[0][0] + idx_2D
    end_idx_2D = maxs_2D[0][1] + idx_2D
    start_2D = x_2D[start_idx_2D]
    end_2D = x_2D[end_idx_2D]




    # FIT CURVE TO BACKGROUND 

    # exclude peak
    x_G_no_peak = np.concatenate((x_G[:start_idx_G], x_G[end_idx_G:]))
    y_G_no_peak = np.concatenate((y_G[:start_idx_G], y_G[end_idx_G:]))

    x_2D_no_peak = np.concatenate((x_2D[:start_idx_2D], x_2D[end_idx_2D:]))
    y_2D_no_peak = np.concatenate((y_2D[:start_idx_2D], y_2D[end_idx_2D:]))

    # degree of polynomial fit to background
    degree3_G = 3
    degree3_2D = 3

    # calculate polynomial
    z3_G = np.polyfit(x_G_no_peak, y_G_no_peak, degree3_G)
    f3_G = np.poly1d(z3_G)

    z3_2D = np.polyfit(x_2D_no_peak, y_2D_no_peak, degree3_2D)
    f3_2D = np.poly1d(z3_2D)



    # SUBTRACT OUT BACKGROUND
    y_G_fixed = y_G - f3_G(x_G)
    y_2D_fixed = y_2D - f3_2D(x_2D)
    
    
    
    # CURVE FITTING
    from scipy.optimize import curve_fit
    from math import pi

    xdata_G = x_G
    ydata_G = y_G_fixed
    xdata_2D = x_2D
    ydata_2D = y_2D_fixed

    # initial parameters (guess - doesn't really matter)
    init_center_G = 1600
    init_width_G = 10
    init_amplitude_factor_G = 300

    init_center_2D = 2700
    init_width_2D = 10
    init_amplitude_factor_2D = 300

    init_params_G = [init_center_G, init_width_G, init_amplitude_factor_G]
    init_params_2D = [init_center_2D, init_width_2D, init_amplitude_factor_2D]

    # lorentzian peak function
    def lorentz(x, center, width, A):
        return ( A * (0.5 * width) / (pi * ( ((x - center) ** 2) + ((0.5 * width) ** 2) ) ))

    params_G, cov_G = curve_fit(lorentz, xdata_G, ydata_G, p0=init_params_G)
    height_G = params_G[2] * 2 / (pi * params_G[1])
    FWHM_G = params_G[1]

    params_2D, cov_2D = curve_fit(lorentz, xdata_2D, ydata_2D, p0=init_params_2D)
    height_2D = params_2D[2] * 2 / (pi * params_2D[1])
    FWHM_2D = params_2D[1]

    ratio = height_2D / height_G
    
    # PLOTS 

    plt.figure(0)
    py.title('Spectrum')
    spec_start = np.argmax(x_np>1000)
    spec_end = np.argmax(x_np>3000)
    py.scatter(x[spec_start:spec_end],y[spec_start:spec_end])
    py.show()

    plt.figure(1)
    py.title('Polynomial Fit: G Peak')
    py.scatter(x_G,y_G)
    py.plot(x_G,y_G_poly,'-r')
    py.show()

    plt.figure(2)
    py.title('Polynomial Fit of Second Derivative: G peak')
    py.scatter(x_G,derivs2_G)
    py.plot(x_G, y2_G,'-r')
    py.show()

    plt.figure(3)
    py.title('Spectrum with Cutoffs : G peak')
    plt.axvline(x=start_G, linestyle=':', color='r')
    plt.axvline(x=end_G, linestyle=':', color='r')
    py.scatter(x_G,y_G)
    py.show()

    plt.figure(4)
    py.title('Curve Fit No Peak: G Peak')
    py.scatter(x_G, y_G)
    py.plot(x_G, f3_G(x_G), linestyle=':', color='r')
    py.show()

    plt.figure(5)
    py.title('Background Subtracted: G Peak')
    py.scatter(x_G, y_G_fixed)
    py.show()

    plt.figure(6)
    py.title('Polynomial Fit: 2D Peak')
    py.scatter(x_2D,y_2D)
    py.plot(x_2D,y_2D_poly,'-r')
    py.show()

    plt.figure(7)
    py.title('Polynomial Fit of Second Derivative: 2D peak')
    py.scatter(x_2D,derivs2_2D)
    py.plot(x_2D, y2_2D,'-r')
    py.show()

    plt.figure(8)
    py.title('Spectrum with Cutoffs: 2D')
    plt.axvline(x=start_2D, linestyle=':', color='r')
    plt.axvline(x=end_2D, linestyle=':', color='r')
    py.scatter(x_2D,y_2D)
    py.show()

    plt.figure(9)
    py.title('Curve Fit No Peak: 2D Peak')
    py.scatter(x_2D, y_2D)
    py.plot(x_2D, f3_2D(x_2D), linestyle=':', color='r')
    py.show()

    plt.figure(10)
    py.title('Background Subtracted: 2D Peak')
    py.scatter(x_2D, y_2D_fixed)
    py.show()
    
    print('G Height : ' + str(height_G))
    print('G FWHM : ' + str(FWHM_G))

    plt.figure(11)
    py.title('G Peak')
    py.scatter(xdata_G, ydata_G)
    py.plot(xdata_G, lorentz(xdata_G, *params_G), color='r')
    py.show()

   
    print('2D Height : ' + str(height_2D))
    print('2D FWHM : ' + str(FWHM_2D))


    plt.figure(12)
    py.title('2D Peak')
    py.scatter(xdata_2D, ydata_2D)
    py.plot(xdata_2D, lorentz(xdata_2D, *params_2D), color='r')
    py.show()

    ratio = height_2D / height_G

    print('2D / G ratio : ' + str(ratio))


    return([ratio, height_G, height_2D])